In [1]:
%cd /home/slimhy/Documents/PADS/code
from datasets.part_occupancies import PartOccupancyDataset, collate
import plotly.graph_objects as go
import numpy as np
import torch


def viz_queries(data_tuple, sample_idx=0):
    """Visualize query points colored by occupancy and bounding box corners"""
    # Get query points and labels
    query_points = data_tuple['query_points'][sample_idx].cpu().numpy()  # [N, 3]
    query_labels = data_tuple['query_labels'][sample_idx].cpu().numpy()  # [N]
    
    # Get bounding box corners
    bbs = data_tuple['part_bbs'][sample_idx].cpu().numpy()  # [P, 8, 3]
    bb_points = bbs.reshape(-1, 3)  # [P*8, 3]
    
    # Split query points by label
    inside_mask = query_labels == 1
    inside_points = query_points[inside_mask]
    outside_points = query_points[~inside_mask]
    
    fig = go.Figure(data=[
        # Inside query points (red)
        go.Scatter3d(
            x=inside_points[:, 0] if len(inside_points) > 0 else [],
            y=inside_points[:, 1] if len(inside_points) > 0 else [],
            z=inside_points[:, 2] if len(inside_points) > 0 else [],
            mode='markers',
            marker=dict(
                size=2,
                color='red',
                opacity=0.8
            ),
            name='Inside Points'
        ),
        # Outside query points (green)
        go.Scatter3d(
            x=outside_points[:, 0] if len(outside_points) > 0 else [],
            y=outside_points[:, 1] if len(outside_points) > 0 else [],
            z=outside_points[:, 2] if len(outside_points) > 0 else [],
            mode='markers',
            marker=dict(
                size=2,
                color='green',
                opacity=0.8
            ),
            name='Outside Points'
        ),
        # Bounding box corners (blue)
        go.Scatter3d(
            x=bb_points[:, 0],
            y=bb_points[:, 1],
            z=bb_points[:, 2],
            mode='markers',
            marker=dict(
                size=8,
                color='blue',
                symbol='square',
                opacity=1.0
            ),
            name='Box Corners'
        )
    ])
    
    fig.update_layout(
        title='Query Points and Box Corners',
        scene=dict(
            aspectmode='data',
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        showlegend=True,
        width=800,
        height=800
    )
    
    return fig

/home/slimhy/Documents/PADS/code


In [2]:
# Create dataset and dataloader
dataset = PartOccupancyDataset(
    hdf5_path="/home/slimhy/Documents/PADS/data/dataset__debug.h5",
    split="train",
    num_queries=2048,
    num_part_points=1024,
)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    num_workers=0,
    collate_fn=collate
)

In [5]:
# Example usage:
data_tuple = next(iter(dataloader))

In [8]:
fig = viz_queries(data_tuple)
fig.show()

In [1]:
import numpy as np

def demonstrate_fp16_effects():
    # Create arrays with different numeric ranges
    normal_range = np.array([1.0, 1.1, 1.2, 1.3], dtype=np.float32)
    small_numbers = np.array([1e-5, 2e-5, 3e-5, 4e-5], dtype=np.float32)
    large_numbers = np.array([65504.0, 65505.0, 65506.0, 65507.0], dtype=np.float32)
    
    # Convert to fp16 and back to fp32 to show precision effects
    normal_fp16 = normal_range.astype(np.float16).astype(np.float32)
    small_fp16 = small_numbers.astype(np.float16).astype(np.float32)
    large_fp16 = large_numbers.astype(np.float16).astype(np.float32)
    
    print("1. Normal Range Numbers:")
    print("Original (fp32):", normal_range)
    print("After fp16:    ", normal_fp16)
    print("Absolute Error:", np.abs(normal_range - normal_fp16))
    print("\n2. Small Numbers (Near Subnormal Range):")
    print("Original (fp32):", small_numbers)
    print("After fp16:    ", small_fp16)
    print("Absolute Error:", np.abs(small_numbers - small_fp16))
    print("\n3. Large Numbers (Near Max Range):")
    print("Original (fp32):", large_numbers)
    print("After fp16:    ", large_fp16)
    print("Absolute Error:", np.abs(large_numbers - large_fp16))
    
    # Demonstrate subnormal numbers
    subnormals = np.array([5.96e-8, 5.97e-8, 5.98e-8, 5.99e-8], dtype=np.float32)
    subnormals_fp16 = subnormals.astype(np.float16).astype(np.float32)
    print("\n4. Subnormal Numbers:")
    print("Original (fp32):", subnormals)
    print("After fp16:    ", subnormals_fp16)
    print("Absolute Error:", np.abs(subnormals - subnormals_fp16))
    
    # Demonstrate spacing between representable numbers
    spacing_test = np.array([10.0, 10.0625, 10.125, 10.1875], dtype=np.float32)
    spacing_fp16 = spacing_test.astype(np.float16).astype(np.float32)
    print("\n5. Spacing Between Numbers around 10:")
    print("Original (fp32):", spacing_test)
    print("After fp16:    ", spacing_fp16)
    print("Absolute Error:", np.abs(spacing_test - spacing_fp16))

demonstrate_fp16_effects()

1. Normal Range Numbers:
Original (fp32): [1.  1.1 1.2 1.3]
After fp16:     [1.        1.0996094 1.2001953 1.2998047]
Absolute Error: [0.         0.00039065 0.00019526 0.00019526]

2. Small Numbers (Near Subnormal Range):
Original (fp32): [1.e-05 2.e-05 3.e-05 4.e-05]
After fp16:     [1.0013580e-05 2.0027161e-05 2.9981136e-05 3.9994717e-05]
Absolute Error: [1.3580575e-08 2.7161150e-08 1.8862920e-08 5.2823452e-09]

3. Large Numbers (Near Max Range):
Original (fp32): [65504. 65505. 65506. 65507.]
After fp16:     [65504. 65504. 65504. 65504.]
Absolute Error: [0. 1. 2. 3.]

4. Subnormal Numbers:
Original (fp32): [5.96e-08 5.97e-08 5.98e-08 5.99e-08]
After fp16:     [5.9604645e-08 5.9604645e-08 5.9604645e-08 5.9604645e-08]
Absolute Error: [4.6433968e-12 9.5354835e-11 1.9535662e-10 2.9535840e-10]

5. Spacing Between Numbers around 10:
Original (fp32): [10.     10.0625 10.125  10.1875]
After fp16:     [10.     10.0625 10.125  10.1875]
Absolute Error: [0. 0. 0. 0.]
